# Module 04: Time Series Visualization

**Estimated Time**: 90 minutes  
**Difficulty**: Intermediate

## Learning Objectives

By the end of this module, you will:
- Handle and manipulate datetime data in Python
- Create temporal line plots with proper date formatting
- Visualize trends, seasonality, and patterns
- Plot multiple time series for comparison
- Use rolling averages and resampling
- Create professional financial and weather visualizations
- Handle missing data in time series

---

In [ ]:
# Import required libraries
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import warnings

# Set style
sns.set_theme(style="whitegrid", context="notebook")
warnings.filterwarnings("ignore")

# Set random seed for reproducibility
np.random.seed(42)

print("Libraries imported successfully!")

## Part 1: Understanding DateTime Data

Time series data has a temporal component - measurements taken at specific points in time.

### Common Time Series Applications
- **Finance**: Stock prices, cryptocurrency, trading volumes
- **Weather**: Temperature, precipitation, humidity
- **Business**: Sales, website traffic, user engagement
- **IoT**: Sensor readings, system metrics
- **Health**: Heart rate, blood pressure, activity levels

### DateTime Objects in Python
- `datetime`: Built-in Python module
- `pd.Timestamp`: Pandas datetime (more flexible)
- `pd.DatetimeIndex`: Index for time series DataFrames

In [ ]:
# Example 1: Creating datetime objects
# Python datetime
dt = datetime(2024, 1, 15, 14, 30)
print(f"Python datetime: {dt}")
print(f"Type: {type(dt)}")

# Pandas Timestamp
ts = pd.Timestamp("2024-01-15 14:30:00")
print(f"\nPandas Timestamp: {ts}")
print(f"Type: {type(ts)}")

# Date range
date_range = pd.date_range(start="2024-01-01", end="2024-01-10", freq="D")
print(f"\nDate range (10 days):")
print(date_range)

print(f"\nFrequencies available:")
print("  'D' = Daily")
print("  'W' = Weekly")
print("  'M' = Month end")
print("  'H' = Hourly")
print("  'T' or 'min' = Minute")
print("  'S' = Second")

In [ ]:
# Example 2: Creating a time series DataFrame
dates = pd.date_range(start="2024-01-01", periods=365, freq="D")

# Simulate daily temperature data with seasonal pattern
day_of_year = np.arange(365)
seasonal_pattern = 15 * np.sin(2 * np.pi * day_of_year / 365 - np.pi / 2) + 20
daily_variation = np.random.normal(0, 3, 365)
temperature = seasonal_pattern + daily_variation

weather_df = pd.DataFrame(
    {"date": dates, "temperature": temperature, "humidity": np.random.uniform(40, 80, 365)}
)

# Set date as index (important for time series!)
weather_df.set_index("date", inplace=True)

print("Weather DataFrame:")
print(weather_df.head(10))
print(f"\nIndex type: {type(weather_df.index)}")
print(f"Frequency: {weather_df.index.freq}")

In [ ]:
# Example 3: Parsing dates from strings
# Common date formats
date_strings = [
    "2024-01-15",  # ISO format
    "01/15/2024",  # US format
    "15-01-2024",  # European format
    "January 15, 2024",  # Text format
    "2024-01-15 14:30:00",  # With time
]

print("Parsing different date formats:")
for date_str in date_strings:
    parsed = pd.to_datetime(date_str)
    print(f"{date_str:30} → {parsed}")

# Handling custom formats
custom_format = "15-Jan-2024"
parsed_custom = pd.to_datetime(custom_format, format="%d-%b-%Y")
print(f"\nCustom format: {custom_format} → {parsed_custom}")

## Part 2: Basic Time Series Plots

Line plots are the foundation of time series visualization.

In [ ]:
# Example 1: Simple time series plot
fig, ax = plt.subplots(figsize=(14, 6))

ax.plot(weather_df.index, weather_df["temperature"], linewidth=1.5, color="orangered", alpha=0.8)

ax.set_title("Daily Temperature Over One Year", fontsize=16, fontweight="bold")
ax.set_xlabel("Date", fontsize=12)
ax.set_ylabel("Temperature (°C)", fontsize=12)
ax.grid(True, alpha=0.3)

# Format x-axis dates
ax.xaxis.set_major_formatter(mdates.DateFormatter("%b %Y"))
ax.xaxis.set_major_locator(mdates.MonthLocator(interval=1))
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

print("Notice the seasonal pattern - warmer in summer, cooler in winter")

In [ ]:
# Example 2: Multiple time series on same plot
fig, ax = plt.subplots(figsize=(14, 6))

# Plot temperature
ax.plot(
    weather_df.index,
    weather_df["temperature"],
    linewidth=2,
    color="orangered",
    label="Temperature (°C)",
    alpha=0.8,
)

# Create second y-axis for humidity
ax2 = ax.twinx()
ax2.plot(
    weather_df.index,
    weather_df["humidity"],
    linewidth=2,
    color="steelblue",
    label="Humidity (%)",
    alpha=0.6,
)

# Customize axes
ax.set_xlabel("Date", fontsize=12, fontweight="bold")
ax.set_ylabel("Temperature (°C)", fontsize=12, color="orangered", fontweight="bold")
ax2.set_ylabel("Humidity (%)", fontsize=12, color="steelblue", fontweight="bold")

ax.set_title("Temperature and Humidity Over Time (Dual Axes)", fontsize=16, fontweight="bold")

# Format dates
ax.xaxis.set_major_formatter(mdates.DateFormatter("%b"))
ax.xaxis.set_major_locator(mdates.MonthLocator(interval=1))
plt.xticks(rotation=45)

# Combine legends
lines1, labels1 = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax.legend(lines1 + lines2, labels1 + labels2, loc="upper left", fontsize=11)

ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("Dual axes allow comparing variables with different scales!")

In [ ]:
# Example 3: Highlighting specific periods
fig, ax = plt.subplots(figsize=(14, 6))

ax.plot(weather_df.index, weather_df["temperature"], linewidth=2, color="navy", alpha=0.8)

# Highlight summer months (June-August)
summer_start = pd.Timestamp("2024-06-01")
summer_end = pd.Timestamp("2024-08-31")
ax.axvspan(summer_start, summer_end, alpha=0.2, color="yellow", label="Summer")

# Highlight winter months (December-February)
winter_start = pd.Timestamp("2024-12-01")
winter_end = pd.Timestamp("2024-12-31")
ax.axvspan(winter_start, winter_end, alpha=0.2, color="lightblue", label="Winter")

# Add mean line
mean_temp = weather_df["temperature"].mean()
ax.axhline(
    y=mean_temp,
    color="red",
    linestyle="--",
    linewidth=2,
    label=f"Annual Mean: {mean_temp:.1f}°C",
    alpha=0.7,
)

ax.set_title(
    "Temperature Throughout the Year with Seasonal Highlights", fontsize=16, fontweight="bold"
)
ax.set_xlabel("Date", fontsize=12)
ax.set_ylabel("Temperature (°C)", fontsize=12)
ax.legend(loc="upper right", fontsize=11)
ax.grid(True, alpha=0.3)

# Format dates
ax.xaxis.set_major_formatter(mdates.DateFormatter("%b"))
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

## Part 3: Resampling and Aggregation

Resampling changes the frequency of time series data.

### Common Operations
- **Downsampling**: Higher to lower frequency (daily → monthly)
- **Upsampling**: Lower to higher frequency (monthly → daily)
- **Aggregation**: mean(), sum(), max(), min(), etc.

In [ ]:
# Example 1: Resampling to different frequencies
# Weekly averages
weekly_avg = weather_df["temperature"].resample("W").mean()

# Monthly averages
monthly_avg = weather_df["temperature"].resample("M").mean()

fig, axes = plt.subplots(3, 1, figsize=(14, 12))

# Daily data
axes[0].plot(weather_df.index, weather_df["temperature"], linewidth=1, color="gray", alpha=0.5)
axes[0].set_title("Daily Temperature", fontsize=14, fontweight="bold")
axes[0].set_ylabel("Temperature (°C)", fontsize=11)
axes[0].grid(True, alpha=0.3)

# Weekly averages
axes[1].plot(weekly_avg.index, weekly_avg, linewidth=2, color="blue", marker="o", markersize=4)
axes[1].set_title("Weekly Average Temperature", fontsize=14, fontweight="bold")
axes[1].set_ylabel("Temperature (°C)", fontsize=11)
axes[1].grid(True, alpha=0.3)

# Monthly averages
axes[2].plot(monthly_avg.index, monthly_avg, linewidth=2.5, color="red", marker="s", markersize=6)
axes[2].set_title("Monthly Average Temperature", fontsize=14, fontweight="bold")
axes[2].set_xlabel("Date", fontsize=11)
axes[2].set_ylabel("Temperature (°C)", fontsize=11)
axes[2].grid(True, alpha=0.3)

# Format dates for all subplots
for ax in axes:
    ax.xaxis.set_major_formatter(mdates.DateFormatter("%b"))
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=45)

fig.suptitle("Temperature at Different Time Scales", fontsize=16, fontweight="bold", y=0.995)
plt.tight_layout()
plt.show()

print("Resampling reveals different patterns at different time scales")

In [ ]:
# Example 2: Different aggregation functions
monthly_stats = (
    weather_df["temperature"]
    .resample("M")
    .agg([("mean", "mean"), ("min", "min"), ("max", "max"), ("std", "std")])
)

fig, ax = plt.subplots(figsize=(14, 7))

# Plot mean with error band (std)
ax.plot(
    monthly_stats.index,
    monthly_stats["mean"],
    linewidth=2.5,
    color="navy",
    label="Mean",
    marker="o",
    markersize=7,
)

# Add error band (±1 std)
ax.fill_between(
    monthly_stats.index,
    monthly_stats["mean"] - monthly_stats["std"],
    monthly_stats["mean"] + monthly_stats["std"],
    alpha=0.3,
    color="navy",
    label="±1 Std Dev",
)

# Plot min and max
ax.plot(
    monthly_stats.index,
    monthly_stats["min"],
    linewidth=1.5,
    color="blue",
    linestyle="--",
    label="Min",
    marker="v",
    markersize=5,
)
ax.plot(
    monthly_stats.index,
    monthly_stats["max"],
    linewidth=1.5,
    color="red",
    linestyle="--",
    label="Max",
    marker="^",
    markersize=5,
)

ax.set_title("Monthly Temperature Statistics", fontsize=16, fontweight="bold")
ax.set_xlabel("Month", fontsize=12)
ax.set_ylabel("Temperature (°C)", fontsize=12)
ax.legend(loc="upper left", fontsize=11)
ax.grid(True, alpha=0.3)

ax.xaxis.set_major_formatter(mdates.DateFormatter("%b"))
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

## Part 4: Rolling Windows and Moving Averages

Rolling windows smooth noisy data and reveal trends.

### Common Uses
- Smooth volatile data (stock prices)
- Identify trends
- Remove noise
- Calculate rolling statistics

In [ ]:
# Example 1: Moving averages with different window sizes
# Calculate rolling means
roll_7 = weather_df["temperature"].rolling(window=7).mean()
roll_30 = weather_df["temperature"].rolling(window=30).mean()
roll_90 = weather_df["temperature"].rolling(window=90).mean()

fig, ax = plt.subplots(figsize=(14, 7))

# Original data
ax.plot(
    weather_df.index,
    weather_df["temperature"],
    linewidth=0.8,
    color="gray",
    alpha=0.4,
    label="Daily Temperature",
)

# Moving averages
ax.plot(roll_7.index, roll_7, linewidth=1.5, color="blue", label="7-day MA", alpha=0.8)
ax.plot(roll_30.index, roll_30, linewidth=2, color="green", label="30-day MA", alpha=0.8)
ax.plot(roll_90.index, roll_90, linewidth=2.5, color="red", label="90-day MA", alpha=0.8)

ax.set_title("Temperature with Moving Averages", fontsize=16, fontweight="bold")
ax.set_xlabel("Date", fontsize=12)
ax.set_ylabel("Temperature (°C)", fontsize=12)
ax.legend(loc="upper left", fontsize=11)
ax.grid(True, alpha=0.3)

ax.xaxis.set_major_formatter(mdates.DateFormatter("%b"))
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

print("Longer windows = smoother curves, but more lag")
print("Shorter windows = more responsive to changes, but noisier")

In [ ]:
# Example 2: Rolling statistics
window = 30

rolling_mean = weather_df["temperature"].rolling(window=window).mean()
rolling_std = weather_df["temperature"].rolling(window=window).std()
rolling_min = weather_df["temperature"].rolling(window=window).min()
rolling_max = weather_df["temperature"].rolling(window=window).max()

fig, ax = plt.subplots(figsize=(14, 7))

# Plot mean
ax.plot(rolling_mean.index, rolling_mean, linewidth=2.5, color="navy", label="30-day Mean")

# Plot confidence band (mean ± 2*std)
ax.fill_between(
    rolling_mean.index,
    rolling_mean - 2 * rolling_std,
    rolling_mean + 2 * rolling_std,
    alpha=0.2,
    color="navy",
    label="±2 Std Dev (95% CI)",
)

# Plot rolling min/max
ax.plot(
    rolling_min.index, rolling_min, linewidth=1, color="blue", linestyle=":", label="30-day Min"
)
ax.plot(rolling_max.index, rolling_max, linewidth=1, color="red", linestyle=":", label="30-day Max")

ax.set_title("30-Day Rolling Statistics", fontsize=16, fontweight="bold")
ax.set_xlabel("Date", fontsize=12)
ax.set_ylabel("Temperature (°C)", fontsize=12)
ax.legend(loc="upper left", fontsize=11)
ax.grid(True, alpha=0.3)

ax.xaxis.set_major_formatter(mdates.DateFormatter("%b"))
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

## Part 5: Financial Data Visualization

Financial time series have special visualization needs.

### Common Financial Plots
- Line charts for price trends
- Candlestick charts (OHLC data)
- Volume bars
- Returns and volatility

In [ ]:
# Create synthetic stock data
dates = pd.date_range(start="2023-01-01", end="2024-12-31", freq="D")

# Simulate stock price with random walk
np.random.seed(42)
returns = np.random.normal(0.0005, 0.02, len(dates))
price = 100 * np.exp(np.cumsum(returns))

# Create volume data
volume = np.random.lognormal(15, 0.5, len(dates))

stock_df = pd.DataFrame({"date": dates, "price": price, "volume": volume})
stock_df.set_index("date", inplace=True)

print("Stock data created:")
print(stock_df.head())
print(f"\nDate range: {stock_df.index.min()} to {stock_df.index.max()}")
print(f"Price range: ${stock_df['price'].min():.2f} to ${stock_df['price'].max():.2f}")

In [ ]:
# Example 1: Stock price with volume
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10), gridspec_kw={"height_ratios": [3, 1]})

# Price plot
ax1.plot(stock_df.index, stock_df["price"], linewidth=1.5, color="darkblue", label="Stock Price")

# Add moving averages
ma_50 = stock_df["price"].rolling(window=50).mean()
ma_200 = stock_df["price"].rolling(window=200).mean()

ax1.plot(ma_50.index, ma_50, linewidth=2, color="orange", label="50-day MA", alpha=0.8)
ax1.plot(ma_200.index, ma_200, linewidth=2, color="red", label="200-day MA", alpha=0.8)

ax1.set_title("Stock Price with Moving Averages and Volume", fontsize=16, fontweight="bold")
ax1.set_ylabel("Price ($)", fontsize=12, fontweight="bold")
ax1.legend(loc="upper left", fontsize=11)
ax1.grid(True, alpha=0.3)

# Volume bars
ax2.bar(stock_df.index, stock_df["volume"], width=1, color="steelblue", alpha=0.6)
ax2.set_ylabel("Volume", fontsize=12, fontweight="bold")
ax2.set_xlabel("Date", fontsize=12, fontweight="bold")
ax2.grid(True, alpha=0.3, axis="y")

# Format dates
for ax in [ax1, ax2]:
    ax.xaxis.set_major_formatter(mdates.DateFormatter("%b %Y"))
    ax.xaxis.set_major_locator(mdates.MonthLocator(interval=3))
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=45)

plt.tight_layout()
plt.show()

print("Technical analysis insights:")
print("- Price above 50-day MA = Short-term bullish")
print("- Price above 200-day MA = Long-term bullish")
print("- Golden cross (50 crosses above 200) = Strong buy signal")

In [ ]:
# Example 2: Returns and volatility
# Calculate daily returns
stock_df["returns"] = stock_df["price"].pct_change() * 100  # percentage

# Calculate rolling volatility (30-day)
stock_df["volatility"] = stock_df["returns"].rolling(window=30).std()

fig, axes = plt.subplots(3, 1, figsize=(14, 12))

# Price
axes[0].plot(stock_df.index, stock_df["price"], linewidth=1.5, color="navy")
axes[0].set_title("Stock Price", fontsize=14, fontweight="bold")
axes[0].set_ylabel("Price ($)", fontsize=11)
axes[0].grid(True, alpha=0.3)

# Daily returns
colors = ["green" if x > 0 else "red" for x in stock_df["returns"]]
axes[1].bar(stock_df.index, stock_df["returns"], color=colors, alpha=0.6, width=1)
axes[1].axhline(y=0, color="black", linewidth=0.8)
axes[1].set_title("Daily Returns", fontsize=14, fontweight="bold")
axes[1].set_ylabel("Returns (%)", fontsize=11)
axes[1].grid(True, alpha=0.3)

# Volatility
axes[2].plot(stock_df.index, stock_df["volatility"], linewidth=2, color="purple")
axes[2].fill_between(stock_df.index, stock_df["volatility"], alpha=0.3, color="purple")
axes[2].set_title("30-Day Rolling Volatility", fontsize=14, fontweight="bold")
axes[2].set_ylabel("Volatility (%)", fontsize=11)
axes[2].set_xlabel("Date", fontsize=11)
axes[2].grid(True, alpha=0.3)

# Format dates
for ax in axes:
    ax.xaxis.set_major_formatter(mdates.DateFormatter("%b %Y"))
    ax.xaxis.set_major_locator(mdates.MonthLocator(interval=3))
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=45)

fig.suptitle(
    "Stock Analysis: Price, Returns, and Volatility", fontsize=16, fontweight="bold", y=0.995
)
plt.tight_layout()
plt.show()

print(f"Average daily return: {stock_df['returns'].mean():.3f}%")
print(f"Average volatility: {stock_df['volatility'].mean():.3f}%")
print(f"Best day: {stock_df['returns'].max():.2f}%")
print(f"Worst day: {stock_df['returns'].min():.2f}%")

## Part 6: Comparing Multiple Time Series

Often you need to compare multiple time series simultaneously.

In [ ]:
# Create multiple stock data
np.random.seed(42)
dates = pd.date_range(start="2023-01-01", end="2024-12-31", freq="D")

# Simulate 4 different stocks
stocks = {}
for stock, (drift, vol) in [
    ("TechCo", (0.001, 0.025)),
    ("FinanceCorp", (0.0005, 0.015)),
    ("EnergyCo", (0.0003, 0.03)),
    ("RetailInc", (0.0007, 0.02)),
]:
    returns = np.random.normal(drift, vol, len(dates))
    stocks[stock] = 100 * np.exp(np.cumsum(returns))

multi_stock_df = pd.DataFrame(stocks, index=dates)

print("Multiple stocks data:")
print(multi_stock_df.head())

In [ ]:
# Example 1: Multiple lines on same plot
fig, ax = plt.subplots(figsize=(14, 7))

colors = ["blue", "red", "green", "purple"]
for (stock, data), color in zip(multi_stock_df.items(), colors):
    ax.plot(multi_stock_df.index, data, linewidth=2, label=stock, color=color, alpha=0.8)

ax.set_title("Stock Prices Comparison (Absolute Values)", fontsize=16, fontweight="bold")
ax.set_xlabel("Date", fontsize=12)
ax.set_ylabel("Price ($)", fontsize=12)
ax.legend(loc="upper left", fontsize=11)
ax.grid(True, alpha=0.3)

ax.xaxis.set_major_formatter(mdates.DateFormatter("%b %Y"))
ax.xaxis.set_major_locator(mdates.MonthLocator(interval=3))
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Example 2: Normalized comparison (percent change from start)
# Normalize all stocks to 100 at start
normalized_df = multi_stock_df / multi_stock_df.iloc[0] * 100

fig, ax = plt.subplots(figsize=(14, 7))

colors = ["blue", "red", "green", "purple"]
for (stock, data), color in zip(normalized_df.items(), colors):
    ax.plot(normalized_df.index, data, linewidth=2, label=stock, color=color, alpha=0.8)

# Add reference line at 100
ax.axhline(y=100, color="black", linestyle="--", linewidth=1, alpha=0.5)

ax.set_title("Stock Prices Comparison (Normalized to 100)", fontsize=16, fontweight="bold")
ax.set_xlabel("Date", fontsize=12)
ax.set_ylabel("Normalized Price (Start = 100)", fontsize=12)
ax.legend(loc="upper left", fontsize=11)
ax.grid(True, alpha=0.3)

ax.xaxis.set_major_formatter(mdates.DateFormatter("%b %Y"))
ax.xaxis.set_major_locator(mdates.MonthLocator(interval=3))
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

print("Normalized view shows relative performance:")
for stock in normalized_df.columns:
    change = ((normalized_df[stock].iloc[-1] / 100) - 1) * 100
    print(f"{stock:15} {change:+6.1f}%")

In [ ]:
# Example 3: Small multiples (faceted plots)
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.ravel()

colors = ["blue", "red", "green", "purple"]

for ax, (stock, data), color in zip(axes, multi_stock_df.items(), colors):
    # Plot price
    ax.plot(multi_stock_df.index, data, linewidth=2, color=color, alpha=0.8)

    # Add 50-day MA
    ma_50 = data.rolling(window=50).mean()
    ax.plot(
        ma_50.index,
        ma_50,
        linewidth=1.5,
        color="orange",
        linestyle="--",
        alpha=0.7,
        label="50-day MA",
    )

    ax.set_title(stock, fontsize=14, fontweight="bold")
    ax.set_ylabel("Price ($)", fontsize=11)
    ax.legend(loc="upper left", fontsize=9)
    ax.grid(True, alpha=0.3)

    # Format dates
    ax.xaxis.set_major_formatter(mdates.DateFormatter("%b %y"))
    ax.xaxis.set_major_locator(mdates.MonthLocator(interval=6))
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=45)

fig.suptitle("Individual Stock Analysis", fontsize=16, fontweight="bold")
plt.tight_layout()
plt.show()

print("Small multiples allow detailed comparison of individual patterns")

## Part 7: Handling Missing Data and Gaps

Real-world time series often have missing data.

In [ ]:
# Create data with missing values
dates = pd.date_range(start="2024-01-01", periods=100, freq="D")
values = np.random.randn(100).cumsum() + 100

# Randomly remove some values
np.random.seed(42)
missing_indices = np.random.choice(100, 15, replace=False)
values[missing_indices] = np.nan

incomplete_df = pd.DataFrame({"value": values}, index=dates)

print(f"Total points: {len(incomplete_df)}")
print(f"Missing points: {incomplete_df['value'].isna().sum()}")
print(f"\nFirst few rows with missing data:")
print(incomplete_df[incomplete_df["value"].isna()].head())

In [ ]:
# Example 1: Different strategies for handling missing data
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Original with gaps
axes[0, 0].plot(
    incomplete_df.index, incomplete_df["value"], linewidth=2, marker="o", markersize=4, color="blue"
)
axes[0, 0].set_title("Original Data (with gaps)", fontsize=14, fontweight="bold")
axes[0, 0].set_ylabel("Value", fontsize=11)
axes[0, 0].grid(True, alpha=0.3)

# Forward fill
filled_forward = incomplete_df["value"].fillna(method="ffill")
axes[0, 1].plot(incomplete_df.index, filled_forward, linewidth=2, color="green")
axes[0, 1].scatter(
    incomplete_df.index[missing_indices],
    filled_forward[missing_indices],
    color="red",
    s=50,
    zorder=5,
    label="Filled values",
)
axes[0, 1].set_title("Forward Fill", fontsize=14, fontweight="bold")
axes[0, 1].set_ylabel("Value", fontsize=11)
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Linear interpolation
filled_interp = incomplete_df["value"].interpolate(method="linear")
axes[1, 0].plot(incomplete_df.index, filled_interp, linewidth=2, color="orange")
axes[1, 0].scatter(
    incomplete_df.index[missing_indices],
    filled_interp[missing_indices],
    color="red",
    s=50,
    zorder=5,
    label="Interpolated",
)
axes[1, 0].set_title("Linear Interpolation", fontsize=14, fontweight="bold")
axes[1, 0].set_ylabel("Value", fontsize=11)
axes[1, 0].set_xlabel("Date", fontsize=11)
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Drop missing
dropped = incomplete_df["value"].dropna()
axes[1, 1].plot(dropped.index, dropped, linewidth=2, marker="o", markersize=4, color="purple")
axes[1, 1].set_title("Dropped Missing Values", fontsize=14, fontweight="bold")
axes[1, 1].set_ylabel("Value", fontsize=11)
axes[1, 1].set_xlabel("Date", fontsize=11)
axes[1, 1].grid(True, alpha=0.3)

# Format dates
for ax in axes.ravel():
    ax.xaxis.set_major_formatter(mdates.DateFormatter("%b %d"))
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=45)

fig.suptitle("Strategies for Handling Missing Time Series Data", fontsize=16, fontweight="bold")
plt.tight_layout()
plt.show()

print("Strategies for missing data:")
print("1. Forward fill: Use last known value")
print("2. Interpolation: Estimate based on surrounding values")
print("3. Drop: Remove missing points (creates gaps in plot)")
print("4. Rolling mean: Fill with local average (not shown)")

## Part 8: Key Takeaways

### What You've Learned
✓ **DateTime handling**: Creating, parsing, and formatting dates  
✓ **Basic plots**: Line plots with proper date formatting  
✓ **Resampling**: Converting between time frequencies  
✓ **Rolling windows**: Moving averages and statistics  
✓ **Financial data**: Price, returns, volatility visualization  
✓ **Multiple series**: Comparison and normalization techniques  
✓ **Missing data**: Strategies for handling gaps  

### Best Practices for Time Series Visualization

1. **Always use a datetime index** for time series DataFrames
2. **Format dates appropriately** for the time scale
3. **Normalize when comparing** series with different scales
4. **Use moving averages** to smooth noisy data
5. **Highlight important periods** with shading or annotations
6. **Show volume/activity** alongside price data
7. **Handle missing data** explicitly and transparently

### Date Formatting Quick Reference

```python
# Common formats
mdates.DateFormatter('%Y-%m-%d')  # 2024-01-15
mdates.DateFormatter('%b %Y')      # Jan 2024
mdates.DateFormatter('%d/%m/%Y')   # 15/01/2024
mdates.DateFormatter('%Y')         # 2024

# Common locators
mdates.YearLocator()              # Every year
mdates.MonthLocator(interval=3)   # Every 3 months
mdates.DayLocator(interval=7)     # Every week
```

### What's Next
In **Module 05**, you'll learn interactive visualizations with Plotly:
- Plotly Express vs Graph Objects
- Interactive line, scatter, and bar charts
- Hover tooltips and click events
- Animations and sliders
- Exporting interactive HTML dashboards

---

## Exercises

Apply your time series visualization skills!

### Exercise 1: Weather Data Analysis
Create a comprehensive weather analysis:
1. Use the weather_df dataset created earlier
2. Create a figure with 3 subplots:
   - Daily temperature with 7-day and 30-day moving averages
   - Monthly average temperature with min/max bands
   - Seasonal comparison (average temp by season)
3. Add appropriate titles, labels, and legends

In [ ]:
# Your code here

### Exercise 2: Stock Portfolio Analysis
1. Use the multi_stock_df dataset
2. Calculate daily returns for each stock
3. Create a visualization showing:
   - Cumulative returns (normalized to 100)
   - Rolling 30-day volatility for each stock
   - Correlation heatmap of daily returns
4. Identify which stock had the best performance and lowest volatility

In [ ]:
# Your code here

### Exercise 3: Resampling Practice
1. Create hourly data for one week (temperature readings)
2. Resample to show:
   - 6-hour averages
   - Daily averages
   - Daily min and max
3. Visualize all resampling frequencies in subplots
4. Which frequency shows the pattern most clearly?

In [ ]:
# Your code here

### Exercise 4: Missing Data Challenge
1. Create a time series with intentional gaps (remove random 20% of data)
2. Compare 4 different filling strategies:
   - Forward fill
   - Backward fill
   - Linear interpolation
   - Spline interpolation
3. Visualize each method and discuss which is most appropriate

In [ ]:
# Your code here

### Challenge: Create a Financial Dashboard
Build a comprehensive financial analysis dashboard:
1. Create or load stock data for 2 years
2. Create a 4-panel dashboard showing:
   - Price with 50-day and 200-day moving averages
   - Volume bars
   - Daily returns histogram
   - Rolling volatility (30-day)
3. Highlight important events:
   - Golden cross (50-day MA crosses above 200-day MA)
   - Periods of high volatility (>2x average)
4. Add annotations for the highest and lowest prices
5. Save as high-resolution PNG

In [ ]:
# Your code here

---

**Congratulations!** You've mastered time series visualization. You can now handle datetime data, create temporal plots, and analyze trends and patterns in time-based data.

**Next**: Module 05 - Interactive Plotly Visualizations